# 脚本说明：
1.脚本用于对输入的PDB文件（蛋白-配体）进行Cartesian搜索，并进行饱和突变。

2.脚本会生成一个文件，里面是所有突变的PDB文件和所有突变对应的得分。




# 1. 初始化与环境配置 (Cartesian Mode)

In [1]:
import os
import sys
import urllib.request

# --- 1. 定义下载源 ---
base_url = "https://raw.githubusercontent.com/RosettaCommons/rosetta/master/source/scripts/python/public/"

# 完整的文件清单 (本次增加了 r3.py)
files_to_download = {
    "molfile_to_params.py": "molfile_to_params.py",
    
    # 基础包
    "rosetta_py/__init__.py": "rosetta_py/__init__.py",
    
    # IO 模块
    "rosetta_py/io/__init__.py": "rosetta_py/io/__init__.py",
    "rosetta_py/io/mdl_molfile.py": "rosetta_py/io/mdl_molfile.py",
    
    # Utility 模块 (关键修复点)
    "rosetta_py/utility/__init__.py": "rosetta_py/utility/__init__.py",
    "rosetta_py/utility/rankorder.py": "rosetta_py/utility/rankorder.py",
    "rosetta_py/utility/r3.py": "rosetta_py/utility/r3.py"  # <--- 这就是刚才报错缺少的向量模块
}



# --- 2. 补全下载 ---
for local_path, remote_path in files_to_download.items():
    directory = os.path.dirname(local_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    
    url = base_url + remote_path
    
    # 仅下载不存在的文件
    if not os.path.exists(local_path):
        print(f"⬇️ 正在下载: {local_path} ...")
        try:
            urllib.request.urlretrieve(url, local_path)
        except Exception as e:
            print(f"❌ 下载失败 {local_path}: {e}")
    else:
        # 如果是 r3.py 这种刚报错的文件，即使存在(如果是空的)最好也提示一下
        if "r3.py" in local_path:
             print(f"✅ 检查: {local_path} 已存在。")

print("依赖环境修复完毕。\n")



✅ 检查: rosetta_py/utility/r3.py 已存在。
依赖环境修复完毕。



In [2]:
import pyrosetta
from pyrosetta import *
from pyrosetta.rosetta.core.scoring import ScoreType
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import (
    IncludeCurrent, InitializeFromCommandline, RestrictToRepacking, PreventRepacking
)
from pyrosetta.rosetta.core.select.residue_selector import (
    ResidueIndexSelector, NeighborhoodResidueSelector, NotResidueSelector, AndResidueSelector
)
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.pack.task.operation import (
    OperateOnResidueSubset, 
    PreventRepackingRLT, 
    RestrictAbsentCanonicalAASRLT,
    RestrictToRepackingRLT 
)
import pandas as pd
import py3Dmol

#pyrosetta.init("-ex1 -ex2 -use_input_sc -nstruct 1 -beta_cart")
pyrosetta.init("-ex1 -ex2 -use_input_sc -nstruct 1 -beta_cart -extra_res_fa 3OH_PAME.params")
print("PyRosetta Initialized in CARTESIAN Mode.")

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.m1 2025.47+release.8bb54e8a6dc3e1c027e4f028bdace6bd4691c823 2025-11-20T15:59:05] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init.score_function_corrections: [ WARNING ] Flag -beta_nov16 is set but either -weights are also

In [3]:
input_sdf = "ligand_fixed.sdf"
output_prefix = "3OH_PAME"
res_name = "LIG"

if not os.path.exists(input_sdf):
    print(f"❌ 错误: 找不到输入文件 '{input_sdf}'。")
else:
    script_path = "molfile_to_params.py"
    # 执行命令
    command = f"{sys.executable} {script_path} -n {res_name} -p {output_prefix} --clobber {input_sdf}"
    
    print(f"正在执行: {command}\n")
    exit_code = os.system(command)

    if exit_code == 0:
        print("-" * 30)
        print("🎉 成功了！")
        print(f"1. Params 文件: {output_prefix}.params")
        print(f"2. PDB 文件:    {output_prefix}_0001.pdb")
        print("-" * 30)

    else:
        print("❌ 依然报错，请查看上方具体的 Python Traceback。")

正在执行: /opt/miniconda3/envs/pyrosetta_m4/bin/python molfile_to_params.py -n LIG -p 3OH_PAME --clobber ligand_fixed.sdf

Centering ligands at (  -7.665,   -4.173,    0.454)
Atom names contain duplications -- renaming all atoms.
  Aromatic bonds must be identified explicitly --
  alternating single/double bonds (Kekule structure) won't cut it.
  This warning does not apply to you if your molecule really isn't aromatic.
Total naive charge -0.875, desired charge 0.000, offsetting all atoms by 0.017
Average 51.0 atoms (19.0 non-H atoms) per fragment
(Proteins average 15.5 atoms (7.8 non-H atoms) per residue)
Wrote PDB file 3OH_PAME_0001.pdb
Wrote params file 3OH_PAME.params
------------------------------
🎉 成功了！
1. Params 文件: 3OH_PAME.params
2. PDB 文件:    3OH_PAME_0001.pdb
------------------------------


In [4]:
import os

def robust_merge_pdbs_no_h(protein_pdb, ligand_pdb, output_file, ligand_chain_id="X"):
    """
    升级版合并函数：自动去除所有氢原子，解决 fill_missing_atoms 报错。
    """
    import os
    if not os.path.exists(protein_pdb) or not os.path.exists(ligand_pdb):
        print("❌ 错误: 输入文件缺失。")
        return False

    with open(output_file, 'w') as out_f:
        print(f"1. 正在读取蛋白 (去除氢原子)...")
        with open(protein_pdb, 'r') as pf:
            for line in pf:
                if line.startswith("ATOM") or line.startswith("HETATM"):
                    # 过滤原则：原子名称（第13-16列）中第一个字符是H，或者不仅仅是重原子
                    # 更简单的方法：Rosetta 的氢原子通常以 H 开头
                    atom_name = line[12:16].strip()
                    # 这是一个简单的启发式过滤：如果原子名以H开头，且不是 Hg(汞) 等重原子，则过滤
                    # 但为了安全，通常直接过滤掉元素符号为 H 的行（第77-78列），或者依靠原子名
                    element = line[76:78].strip()
                    
                    # 如果 PDB 没写元素符号，就看原子名首字母
                    is_hydrogen = (element == 'H') or (not element and atom_name.startswith('H'))
                    
                    if not is_hydrogen:
                        out_f.write(line)
        
        out_f.write("TER\n")
        
        print(f"2. 正在植入新配体 (去除氢原子)...")
        with open(ligand_pdb, 'r') as lf:
            for line in lf:
                if line.startswith("ATOM") or line.startswith("HETATM"):
                    # 同样修改 Chain ID 并去氢
                    atom_name = line[12:16].strip()
                    element = line[76:78].strip()
                    is_hydrogen = (element == 'H') or (not element and atom_name.startswith('H'))
                    
                    if not is_hydrogen:
                        # 修改链 ID
                        new_line = line[:21] + ligand_chain_id + line[22:]
                        out_f.write(new_line)
                elif line.startswith("CONECT"):
                    # CONECT 行有时会引用氢原子的序号，会导致报错
                    # 最稳妥的方式是：直接丢弃 CONECT，Rosetta 不需要它（除非是非标准连接）
                    # 对于标准配体，params 文件已经定义了连接，不需要 PDB 里的 CONECT
                    pass 
            
    print(f"✅ 净化完成: {output_file} (无氢版)")
    return True

# --- 执行修复 ---
# 重新生成干净的 PDB
robust_merge_pdbs_no_h("bhpmeh-ns-l.pdb", "3OH_PAME_0001.pdb", "final_complex.pdb", ligand_chain_id="X")

1. 正在读取蛋白 (去除氢原子)...
2. 正在植入新配体 (去除氢原子)...
✅ 净化完成: final_complex.pdb (无氢版)


True

# 2. 辅助工具类

In [5]:
def get_pose_index(pose, chain, pdb_resnum):
    """将 PDB 编号 (e.g., Chain A, Residue 120) 转换为 Pose 索引"""
    pdb_info = pose.pdb_info()
    pose_index = pdb_info.pdb2pose(chain, int(pdb_resnum))
    if pose_index == 0:
        raise ValueError(f"Residue {chain}:{pdb_resnum} not found in PDB.")
    return pose_index

def get_pdb_label(pose, pose_index):
    """将 Pose 索引反向转换为 PDB 标签"""
    pdb_info = pose.pdb_info()
    chain = pdb_info.chain(pose_index)
    resnum = pdb_info.number(pose_index)
    aa = pose.residue(pose_index).name1()
    return f"{aa}{resnum} (Chain {chain})"

# 辅助函数：处理二硫键
from pyrosetta.rosetta.core.chemical import DISULFIDE
from pyrosetta.rosetta.core.conformation import break_disulfide

def find_disulfide_partner(pose, res_index):
    res = pose.residue(res_index)
    for i in range(1, res.n_current_residue_connections() + 1):
        partner_idx = res.connected_residue_at_resconn(i)
        if partner_idx == 0 or partner_idx == res_index: continue
        if pose.residue(partner_idx).has_variant_type(DISULFIDE):
            return partner_idx
    return None

# 3. 加载与Cartesian Pre-Relax

In [6]:
import pyrosetta
from pyrosetta import pose_from_pdb

# 1. 再次确保 Init 包含参数文件 (以防万一)
# 注意：如果之前已经 init 过，这里可能不会生效，建议重启 Kernel 后运行
try:
    pyrosetta.init("-ex1 -ex2 -use_input_sc -beta_cart -extra_res_fa 3OH_PAME.params -mute all")
except:
    pass

print("======== 诊断开始 ========")

# --- 测试 A: 只加载蛋白 ---
print("\n👉 测试 A: 尝试单独加载蛋白 (bhpmeh-ns-l.pdb)...")
try:
    # 为了排除杂质干扰，我们使用 cleanATOM 快速清洗一下
    from pyrosetta.toolbox import cleanATOM
    cleanATOM("bhpmeh-ns-l.pdb") # 这会生成 bhpmeh-ns-l.clean.pdb
    
    pose_prot = pose_from_pdb("bhpmeh-ns-l.clean.pdb")
    print(f"✅ 蛋白加载成功！残基数: {pose_prot.total_residue()}")
    protein_ok = True
except Exception as e:
    print(f"❌ 蛋白加载失败！原因: {e}")
    print("   -> 结论: 您的原始蛋白 PDB 结构有严重缺损。")
    protein_ok = False

# --- 测试 B: 只加载配体 ---
print("\n👉 测试 B: 尝试单独加载配体 (3OH_PAME_0001.pdb)...")
try:
    pose_lig = pose_from_pdb("3OH_PAME_0001.pdb")
    print(f"✅ 配体加载成功！残基名: {pose_lig.residue(1).name3()}")
    ligand_ok = True
except Exception as e:
    print(f"❌ 配体加载失败！原因: {e}")
    print("   -> 结论: Params 文件和 PDB 不匹配，或者配体几何构型极差。")
    ligand_ok = False

# --- 测试 C: 如果 AB 都成功，尝试“软结合” ---
if protein_ok and ligand_ok:
    print("\n👉 测试 C: 尝试代码级合并 (Append)...")
    try:
        # 既然物理文件合并报错，我们试试在内存里“接”上去
        # 1. 克隆蛋白
        combined_pose = pose_prot.clone()
        # 2. 获取配体残基
        ligand_res = pose_lig.residue(1)
        # 3. 强行拼接到蛋白末尾 (通过 Jump 连接)
        combined_pose.append_residue_by_jump(ligand_res, 1) # 接在第1个残基上
        
        print(f"✅ 内存合并成功！总残基: {combined_pose.total_residue()}")
        print("   -> 解决方案: 请放弃文件合并，直接使用下面的‘内存合并’代码。")
        
        # 顺便导出这个成功的结构，供后续使用
        combined_pose.dump_pdb("final_complex_fixed.pdb")
        print("   -> 已生成修复后的文件: final_complex_fixed.pdb")
        
    except Exception as e:
        print(f"❌ 内存合并失败: {e}")
        print("   -> 结论: 配体和蛋白在空间上发生了严重的物理重叠（撞车）。")

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.m1 2025.47+release.8bb54e8a6dc3e1c027e4f028bdace6bd4691c823 2025-11-20T15:59:05] retrieved from: http://www.pyrosetta.org
[ WARNING ] Resetting the global tracer options, which have already been set.
[ WARNING ] This will not affect Tracers which are already initialized.
core.init: Checki

In [10]:
# 1. 加载 PDB
input_pdb = "final_complex_fixed.pdb" 
# 如果没有文件，解开下面注释下载测试
# pyrosetta.toolbox.rcsb.download("1UBQ")
# input_pdb = "1UBQ_clean.pdb"

pose = pose_from_pdb(input_pdb)

# --- 关键变更 2: 使用 Cartesian 打分函数 ---
# ref2015_cart 包含了对键长、键角偏差的惩罚项
scorefxn_cart = create_score_function("ref2015_cart")

print(f"Original Energy (ref2015_cart): {scorefxn_cart(pose):.2f} REU")

# 检测是否存在WT_cartesian_relaxed.pdb,如果存在则使用WT_cartesian_relaxed.pdb作为后续突变的基准，如果不存在则进行FastRelax
import os
if os.path.exists("WT_cartesian_relaxed_l.pdb"):
    print("WT_cartesian_relaxed_l.pdb found, using it as the base pose for Cartesian mutations.")
    pose = pose_from_pdb("WT_cartesian_relaxed_l.pdb")
else:
    print("WT_cartesian_relaxed_l.pdb not found, running FastRelax to generate it.")
    relax = FastRelax()
    relax.set_scorefxn(scorefxn_cart)
    relax.cartesian(True) # <--- 开启笛卡尔模式的灵魂开关
    relax.apply(pose)
    
# 2. 执行 Cartesian FastRelax
# 传统的 Relax 只动二面角，这里我们开启 cartesian 模式，允许键长键角微调
# 注意：这通常比普通 Relax 慢 2-3 倍，但对于 ddG 计算至关重要
print("Running Cartesian Pre-Relax (This allows backbone flexing)...")



wt_score = scorefxn_cart(pose)
print(f"Relaxed WT Energy: {wt_score:.2f} REU")

pose.dump_pdb("WT_cartesian_relaxed_l.pdb")

core.import_pose.import_pose: File 'final_complex_fixed.pdb' automatically determined to be of type PDB from contents.
core.io.pose_from_sfr.PoseFromSFRBuilder: Adding undetected upper terminus type to residue 459,  459 A
core.conformation.Conformation: Found disulfide between residues 54 89
core.conformation.Conformation: Found disulfide between residues 239 250
core.conformation.Conformation: Found disulfide between residues 363 446
core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (460)
Original Energy (ref2015_cart): 193.73 REU
WT_cartesian_relaxed_l.pdb found, using it as the base pose for Cartesian mutations.
core.import_pose.import_pose: File 'WT_cartesian_relaxed_l.pdb' automatically determined to be of type PDB from contents.
core.io.pose_from_sfr.PoseFromSFRBuilder: Adding undetected upper terminus type to residue 459,  459 A
core.conformation.Conformation: Found disulfide between residues 54 89
core.conformation.Conformation: Found disul

True

# 4. 核心算法: Cartesian ddG (Mutate - FlexBB - Minimize)

In [11]:
def calculate_ddg_cartesian(wt_pose, chain_id, pdb_resnum, target_aa_one_letter, packing_radius=8.0):
    """
    Cartesian ddG 版本：
    允许突变位点周围的骨架 (Backbone) 和侧链 (Sidechain) 在 XYZ 空间自由移动，
    以缓解空间位阻。
    """
    # 1. 准备工作
    mut_pose = wt_pose.clone()
    res_index = get_pose_index(mut_pose, chain_id, pdb_resnum)
    wt_aa_name = mut_pose.residue(res_index).name1()
    
    print(f"--- Processing {wt_aa_name}{pdb_resnum} -> {target_aa_one_letter} (Cartesian Protocol) ---")

    # 2. 处理二硫键 (同原代码)
    if mut_pose.residue(res_index).has_variant_type(DISULFIDE):
        partner_index = find_disulfide_partner(mut_pose, res_index)
        if partner_index:
            print(f"Action: Breaking disulfide bond with {partner_index}...")
            break_disulfide(mut_pose.conformation(), res_index, partner_index)

    # 3. 突变操作 (使用 PackerTask 进行强制突变)
    # 这一步先仅仅把氨基酸换掉，侧链放进去，不做深度优化
    tf = TaskFactory()
    tf.push_back(InitializeFromCommandline())
    tf.push_back(IncludeCurrent())

    # A. 目标位点：强制突变
    target_selector = ResidueIndexSelector(res_index)
    rlt = RestrictAbsentCanonicalAASRLT()
    rlt.aas_to_keep(target_aa_one_letter)
    tf.push_back(OperateOnResidueSubset(rlt, target_selector))

    # B. 邻居位点：暂时只允许 Repack (稍后在 Minimization 阶段动骨架)
    nbr_selector = NeighborhoodResidueSelector(target_selector, packing_radius, True)
    nbr_only_selector = AndResidueSelector(nbr_selector, NotResidueSelector(target_selector))
    tf.push_back(OperateOnResidueSubset(RestrictToRepackingRLT(), nbr_only_selector))
    
    # C. 其他位点：冻结
    not_nbr_selector = NotResidueSelector(nbr_selector)
    tf.push_back(OperateOnResidueSubset(PreventRepackingRLT(), not_nbr_selector))

    # 执行突变和初步侧链摆放
    print(f"Action: Mutating to {target_aa_one_letter}...")
    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover(scorefxn_cart)
    packer.task_factory(tf)
    packer.apply(mut_pose)

    # 4. Cartesian Minimization (核心差异)
    # 我们需要构建一个 MoveMap，允许邻居范围内的原子(包括骨架)在 XYZ 空间移动
    
    mm = MoveMap()
    mm.set_bb(False)   # 默认关闭所有骨架
    mm.set_chi(False)  # 默认关闭所有侧链
    
    # 增加一段显式的配体检查 (可选，但推荐)
    # 假设配体是链 X 的第 1 个残基
    # ligand_idx = mut_pose.pdb_info().pdb2pose("X", 1) 
    # if ligand_idx > 0 and selection_vector[ligand_idx]:
    #     print("  -> Ligand is in the neighbor list and will be optimized (Chi only).")
    
    # 获取邻居的布尔向量
    selection_vector = nbr_selector.apply(mut_pose)
    
    # 遍历开启邻居的自由度
    for i in range(1, mut_pose.total_residue() + 1):
        if selection_vector[i]:
            mm.set_chi(i, True) # 允许侧链动
            mm.set_bb(i, True)  # <--- 关键：允许骨架动 (Backbone Flexibility)
    
    # 设置 MinMover
    min_mover = pyrosetta.rosetta.protocols.minimization_packing.MinMover()
    min_mover.movemap(mm)
    min_mover.score_function(scorefxn_cart)
    min_mover.min_type("lbfgs_armijo_nonmonotone")
    
    # --- 开启 Cartesian 模式 ---
    min_mover.cartesian(True) 
    
    print("Action: Running Cartesian Minimization (Relaxing strain)...")
    min_mover.apply(mut_pose)
    
    # 5. 计算最终分数
    # 再次确认突变是否成功
    actual_new_aa = mut_pose.residue(res_index).name1()
    if actual_new_aa != target_aa_one_letter:
        print("CRITICAL ERROR: Mutation failed.")
        return None

    mut_score = scorefxn_cart(mut_pose)
    ddg = mut_score - wt_score 
    
    return {
        "Mutation": f"{wt_aa_name}{pdb_resnum}{target_aa_one_letter}",
        "ddG (REU)": ddg,
        "Mutant_Pose": mut_pose
    }

# 5. 运行计算 (Cartesian)

In [12]:
# 示例：将 A链 第 20 号残基突变为 丙氨酸 (A)
target_chain = "A"
target_resnum = 239 
target_amino_acid = "C" 

# 注意：Cartesian 计算通常比普通计算慢一点
result = calculate_ddg_cartesian(pose, target_chain, target_resnum, target_amino_acid)

# 打印报表
print("-" * 40)
print(f"Cartesian ddG Result for {result['Mutation']}")
print(f"ddG: {result['ddG (REU)']:.3f} REU")
print("-" * 40)

if result['ddG (REU)'] < -1.0:
    print("结论: Stabilizing (显著稳定)")
elif result['ddG (REU)'] > 1.0:
    print("结论: Destabilizing (显著破坏)")
else:
    print("结论: Neutral (中性)")

--- Processing C239 -> C (Cartesian Protocol) ---
Action: Breaking disulfide bond with 250...
Action: Mutating to C...
core.pack.task: Packer task: initialize from command line()


core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (460)
core.pack.pack_rotamers: built 402 rotamers at 11 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
Action: Running Cartesian Minimization (Relaxing strain)...
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 54 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 89 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 363 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 446 BRANCH 1
----------------------------------------
Cartesian ddG Result for C239C
ddG: 5.662 REU
----------------------------------------
结论: Destabilizing (显著破坏)


# 6. 交互式可视化 (前后对比)

In [13]:
import py3Dmol
from pyrosetta.rosetta.std import ostringstream

def get_pdb_string(pose):
    oss = ostringstream()
    pose.dump_pdb(oss) 
    return oss.str()

view = py3Dmol.view(width=800, height=400)

# 加载 WT (灰色)
wt_pdb_str = get_pdb_string(pose) 
view.addModel(wt_pdb_str, "pdb")
view.setStyle({'model': 0}, {'cartoon': {'color': 'lightgray', 'opacity': 0.6}})

# 加载 Mutant (蓝色)
if 'Mutant_Pose' in result:
    mutant_pose = result['Mutant_Pose']
    mut_pdb_str = get_pdb_string(mutant_pose)
    view.addModel(mut_pdb_str, "pdb")
    view.setStyle({'model': 1}, {'cartoon': {'color': 'blue'}})

    # 聚焦突变位点
    idx = get_pose_index(pose, target_chain, target_resnum)
    pdb_res_label = str(pose.pdb_info().number(idx))

    # 显示 WT 侧链 (红)
    view.addStyle({'model': 0, 'resi': pdb_res_label, 'chain': target_chain}, 
                  {'stick': {'colorscheme': 'redCarbon'}})

    # 显示 Mutant 侧链 (绿)
    view.addStyle({'model': 1, 'resi': pdb_res_label, 'chain': target_chain}, 
                  {'stick': {'colorscheme': 'greenCarbon'}})

    view.zoomTo({'resi': pdb_res_label, 'chain': target_chain})

    label_text = f"Cartesian Mode: WT:{pose.residue(idx).name1()} -> Mut:{mutant_pose.residue(idx).name1()}"
    view.addLabel(label_text, 
                  {'position': {'x':0, 'y':0, 'z':0}, 'useScreen': True, 'fontColor':'black', 'backgroundColor':'white'})

view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 饱和突变全自动工作流 (Saturation Mutagenesis Pipeline)

1. 继承 Cartesian ddG 高精度协议
2. 多进程并行计算 (自动调用所有 CPU 核心)
3. 自动生成所有突变的 PDB 结构文件
4. 汇总结果为 CSV 报表

In [14]:
%%writefile my_worker_l.py
import os
import pyrosetta
from pyrosetta import *
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import (
    IncludeCurrent, InitializeFromCommandline, RestrictToRepacking, 
    PreventRepacking, OperateOnResidueSubset, RestrictAbsentCanonicalAASRLT, 
    RestrictToRepackingRLT, PreventRepackingRLT
)
from pyrosetta.rosetta.core.select.residue_selector import (
    ResidueIndexSelector, NeighborhoodResidueSelector, 
    NotResidueSelector, AndResidueSelector
)
from pyrosetta.rosetta.protocols.minimization_packing import MinMover, PackRotamersMover
from pyrosetta.rosetta.core.chemical import DISULFIDE
from pyrosetta.rosetta.core.conformation import break_disulfide

def _worker_calculate_mutation(args):
    """
    工作进程：执行单个突变计算
    """
    target_chain, target_resnum, target_aa, config, input_pdb_path = args
    
    # ================= 修正核心 =================
    # 强制初始化逻辑：
    # 不要使用复杂的 options 判断，直接尝试 init。
    # 即使之前初始化过，PyRosetta 通常也只是报个 Warning，或者是抛出 RuntimeError。
    # 我们捕获这个错误即可，确保这一步“尽力而为”地通过。
    try:
        # 这里的参数必须和主程序保持一致
        # pyrosetta.init("-ex1 -ex2 -use_input_sc -beta_cart -nstruct 1 -mute all")
        pyrosetta.init("-ex1 -ex2 -use_input_sc -beta_cart -nstruct 1 -mute all -extra_res_fa 3OH_PAME.params")
    except RuntimeError:
        # 如果报 "already initialized"，说明环境是好的，直接跳过
        pass
    except Exception as e:
        # 其他未知错误，打印出来方便调试
        print(f"Worker Init Warning: {e}")
    # ===========================================

    # 准备数据结构
    result_entry = {
        "Chain": target_chain, "ResNum": target_resnum, "Mutant_AA": target_aa,
        "WT_AA": "", "ddG": 0.0, "Status": "Success", "PDB_File": ""
    }

    # 加载 PDB
    # 如果 init 失败，这里依然会爆 RuntimeError，但现在我们大大增加了 init 成功的概率
    try:
        pose = pose_from_pdb(input_pdb_path)
    except Exception as e:
        result_entry["Status"] = f"Error loading PDB: {e}"
        return result_entry

    scorefxn = create_score_function("ref2015_cart")
    wt_score = scorefxn(pose)
    
    try:
        # --- 1. 定位与预处理 ---
        pdb_info = pose.pdb_info()
        res_index = pdb_info.pdb2pose(target_chain, int(target_resnum))
        if res_index == 0:
            raise ValueError(f"Residue {target_chain}:{target_resnum} not found")
            
        wt_aa_name = pose.residue(res_index).name1()
        result_entry["WT_AA"] = wt_aa_name
        
        if wt_aa_name == target_aa:
            result_entry["Status"] = "Skipped (WT)"
            return result_entry

        # 处理二硫键
        if pose.residue(res_index).has_variant_type(DISULFIDE):
            res = pose.residue(res_index)
            curr_conns = res.n_current_residue_connections()
            for i in range(1, curr_conns + 1):
                partner = res.connected_residue_at_resconn(i)
                if partner != 0 and partner != res_index:
                    if pose.residue(partner).has_variant_type(DISULFIDE):
                        break_disulfide(pose.conformation(), res_index, partner)
                        break

        # --- 2. 突变操作 ---
        tf = TaskFactory()
        tf.push_back(InitializeFromCommandline())
        tf.push_back(IncludeCurrent())
        
        target_selector = ResidueIndexSelector(res_index)
        rlt = RestrictAbsentCanonicalAASRLT()
        rlt.aas_to_keep(target_aa)
        tf.push_back(OperateOnResidueSubset(rlt, target_selector))
        
        nbr_selector = NeighborhoodResidueSelector(target_selector, 8.0, True)
        nbr_only = AndResidueSelector(nbr_selector, NotResidueSelector(target_selector))
        tf.push_back(OperateOnResidueSubset(RestrictToRepackingRLT(), nbr_only))
        
        not_nbr = NotResidueSelector(nbr_selector)
        tf.push_back(OperateOnResidueSubset(PreventRepackingRLT(), not_nbr))
        
        packer = PackRotamersMover(scorefxn)
        packer.task_factory(tf)
        packer.apply(pose)
        
        # --- 3. Cartesian Minimization ---
        mm = MoveMap()
        mm.set_bb(False)
        mm.set_chi(False)
        
        selection = nbr_selector.apply(pose)
        for i in range(1, pose.total_residue() + 1):
            if selection[i]:
                mm.set_chi(i, True)
                mm.set_bb(i, True)
        
        min_mover = MinMover()
        min_mover.movemap(mm)
        min_mover.score_function(scorefxn)
        min_mover.min_type("lbfgs_armijo_nonmonotone")
        min_mover.cartesian(True)
        min_mover.apply(pose)
        
        # --- 4. 结算 ---
        mut_score = scorefxn(pose)
        ddg = mut_score - wt_score
        
        result_entry["ddG"] = ddg
        
        if config["SAVE_PDB"]:
            filename = f"{target_chain}_{target_resnum}_{target_aa}.pdb"
            filepath = os.path.join(config["OUTPUT_DIR"], filename)
            pose.dump_pdb(filepath)
            result_entry["PDB_File"] = filepath
            
    except Exception as e:
        result_entry["Status"] = f"Error: {str(e)}"

    return result_entry

Writing my_worker_l.py


In [17]:
import multiprocessing
import pandas as pd
import os
import time
import pyrosetta
# === 关键修改：从文件导入 worker 函数 ===
import my_worker_l 
from importlib import reload
reload(my_worker_l) # 防止你修改了文件但 notebook 没更新
# ====================================

# 配置保持不变
CONFIG = {
    "INPUT_PDB": "WT_cartesian_relaxed_l.pdb", 
    "OUTPUT_DIR": "saturation_output_l",       
    "TARGETS": [
        {"chain": "A", "resnum": 235},
        {"chain": "A", "resnum": 239}       
    ],
    "N_STRUCT": 1,                           
    "CPU_CORES": max(1, multiprocessing.cpu_count() - 2),
    "SAVE_PDB": True                          
}

def run_saturation_pipeline():
    # ... (前面的检查代码保持不变) ...
    
    # 1. 环境检查
    if not os.path.exists(CONFIG["INPUT_PDB"]):
        print(f"Error: Input file {CONFIG['INPUT_PDB']} not found!")
        return

    if not os.path.exists(CONFIG["OUTPUT_DIR"]):
        os.makedirs(CONFIG["OUTPUT_DIR"])

    # 2. 生成任务
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
    tasks = []
    
    for target in CONFIG["TARGETS"]:
        chain = target["chain"]
        resnum = target["resnum"]
        for aa in amino_acids:
            tasks.append((chain, resnum, aa, CONFIG, CONFIG["INPUT_PDB"]))
            
    print(f"Using {CONFIG['CPU_CORES']} CPU cores...")
    
    # 3. 并行执行
    # === 关键修改：调用 my_worker._worker_calculate_mutation ===
    with multiprocessing.Pool(processes=CONFIG["CPU_CORES"]) as pool:
        results = pool.map(my_worker_l._worker_calculate_mutation, tasks)
    # ========================================================
    
    # 4. 结果汇总
    df = pd.DataFrame(results)
    
    # 简单清洗数据
    df = df[df["Status"] == "Success"]
    csv_filename = os.path.join(CONFIG["OUTPUT_DIR"], "saturation_results.csv")
    df.to_csv(csv_filename, index=False)
    
    print("Done!")
    return df

if __name__ == "__main__":
    if os.path.exists(CONFIG["INPUT_PDB"]):
        df = run_saturation_pipeline()
        print(df.head())

Using 8 CPU cores...
┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.m1 2025.47+release.8bb54e8a6dc3e1c027e4f028bdace6bd4691c823 2025-11-20T15:59:05] retrieved from: http://www.pyrosetta.org
┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                  